Let's start with the block data from the Flashbots transparency dashboard (https://transparency.flashbots.net/) for Q1 2023, which includes useful fields including the proposer's ETH reward, whether the validator used the MEV-boost Flashbots relay, the block number, the Ethereum network gas price, and more.

We define **realized extractable value** as the difference between the block proposer's ETH balance before and after the block execution.

We also have lists of MEV-botted transactions from Q1 2023 (courtesy of [MEV-inspect](https://github.com/flashbots/mev-inspect-py)), which are strategies that MEV bots take advantage of whether by pure revenue profit or order optimization (OO) (see [Flash Boys 2.0](https://arxiv.org/abs/1904.05234)). These transaction lists include liquidations, arbitrages, sandwiches, and sandwiched swaps.

To add useful predictors for our model, we will count how many of each transaction type is in each block, extract the hour of the day that the block was mined, and derive and aggregate some predictors from the gas fee attributes.

In [34]:
import pandas as pd
from datetime import datetime

In [35]:
# Load the datasets
blocks_df = pd.read_csv('block_data_flashbots_dashboard_q1.csv')
sandwiches_df = pd.read_csv('sandwiches.csv')
liquidations_df = pd.read_csv('liquidations.csv')
sandwiched_swaps_df = pd.read_csv('sandwiched_swaps.csv')
arbitrages_df = pd.read_csv('arbitrages.csv')

# Count the number of transactions of each transaction type for each block number
sandwiches_count = sandwiches_df.groupby('block_number').size().reset_index(name='sandwiches_count')
liquidations_count = liquidations_df.groupby('block_number').size().reset_index(name='liquidations_count')
sandwiched_swaps_count = sandwiched_swaps_df.groupby('block_number').size().reset_index(name='sandwiched_swaps_count')
arbitrages_count = arbitrages_df.groupby('block_number').size().reset_index(name='arbitrages_count')

# Merge the count back into the blocks DataFrame
blocks_df = blocks_df.merge(sandwiches_count, on='block_number', how='left')
blocks_df = blocks_df.merge(liquidations_count, on='block_number', how='left')
blocks_df = blocks_df.merge(sandwiched_swaps_count, on='block_number', how='left')
blocks_df = blocks_df.merge(arbitrages_count, on='block_number', how='left')

# Replace NaN with 0 in the count columns
blocks_df['sandwiches_count'] = blocks_df['sandwiches_count'].fillna(0).astype(int)
blocks_df['liquidations_count'] = blocks_df['liquidations_count'].fillna(0).astype(int)
blocks_df['sandwiched_swaps_count'] = blocks_df['sandwiched_swaps_count'].fillna(0).astype(int)
blocks_df['arbitrages_count'] = blocks_df['arbitrages_count'].fillna(0).astype(int)

# Convert the 'block_timestamp' column to datetime and extract the hour
blocks_df['hour_of_day'] = blocks_df['block_timestamp'].apply(
    lambda x: datetime.utcfromtimestamp(x).hour
)

## Add some new potential predictors & analysis variables

# How much of the block gas limit was utilized. Can be a proxy for network congestion during the time the block was mined.
# Ideally the target should be 15 million gas used out of 30 million, and the base fee fluctuates accordingly (EIP 1559)
blocks_df['block_fullness'] = blocks_df['gas_used'] / 30000000

# Although we don't have the priority fee/tip, we can use this as a proxy for the total gas fees paid in the block
blocks_df['total_gas_fees'] = blocks_df['gas_used'] * blocks_df['base_fee_per_gas_in_eth']

# Change in base fee from one block to another indicates a trend in network congestion
blocks_df['base_fee_change'] = blocks_df['base_fee_per_gas_in_eth'].diff()
blocks_df['base_fee_change_percentage'] = (blocks_df['base_fee_change'] / blocks_df['base_fee_per_gas_in_eth'].shift()) * 100
blocks_df.drop(columns=['base_fee_change'], inplace=True)

# Let's see if this indicator means much
blocks_df['network_demand'] = blocks_df['block_fullness'] * blocks_df['base_fee_change_percentage']

# Handle cases where the previous base fee is zero
blocks_df['network_demand'].fillna(0, inplace=True)


In [36]:
# Convert boolean columns to numeric
blocks_df['validator_is_registered_with_relay'] = blocks_df['validator_is_registered_with_relay'].astype(int)
blocks_df['is_fb_builder'] = blocks_df['is_fb_builder'].astype(int)

In [37]:
# Save the updated DataFrame
blocks_df.to_csv('block_data_ready.csv', index=False)

In [38]:
# Remove all columns we won't use in analysis and save to a cleaned csv
blocks_df.drop(columns=['block_number', 'block_hash', 'block_timestamp', 'header_fee_recipient', 'validator_fee_recipient', 'header_fee_recipient_balance_change_in_eth', 'validator_fee_recipient_balance_change_in_eth', 'gas_limit', 'extra_data', 'block_origin', 'builder'], inplace=True)

# Save the cleaned DataFrame
blocks_df.to_csv('block_data_ready_cleaned.csv', index=False)